In [1]:
import os
import requests
import time
import json
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

## get artist ids

In [2]:
path = "parsed_chart_html"
json_files = sorted(os.listdir(path))
print(json_files)

['2005.jsonl', '2006.jsonl', '2007.jsonl', '2008.jsonl', '2009.jsonl', '2010.jsonl', '2011.jsonl', '2012.jsonl', '2013.jsonl', '2014.jsonl', '2015.jsonl', '2016.jsonl', '2017.jsonl', '2018.jsonl', '2019.jsonl', '2020.jsonl', '2021.jsonl', '2022.jsonl', '2023.jsonl']


In [26]:
artist_ids = set()
for json_file in json_files:
    #print(json_file)
    with open(os.path.join(path, json_file), "r") as f:
        for line in tqdm(f):
            doc = json.loads(line)
            #print(doc)
            artists = doc["artists"]
            artist_id_list = [_[0] for _ in artists]
            artist_ids = artist_ids.union(artist_id_list)

    
print("num artist ids: ", len(artist_ids))

5141it [00:00, 108604.68it/s]
5182it [00:00, 103194.77it/s]
5200it [00:00, 74421.90it/s]
5200it [00:00, 64967.53it/s]
5299it [00:00, 50461.50it/s]
5192it [00:00, 40166.79it/s]
5184it [00:00, 38192.93it/s]
5193it [00:00, 35484.44it/s]
5200it [00:00, 40812.76it/s]
5200it [00:00, 38613.52it/s]
5300it [00:00, 35028.15it/s]
5198it [00:00, 39904.15it/s]
5200it [00:00, 38013.87it/s]
5200it [00:00, 22693.76it/s]
5200it [00:00, 21541.99it/s]
5300it [00:00, 19217.74it/s]
5200it [00:00, 18996.67it/s]
5200it [00:00, 20145.81it/s]
2000it [00:00, 21304.02it/s]

num artist ids:  2305


In [27]:
artist_ids = sorted(artist_ids)

## crawl artist ids

In [28]:
def initBrowser():
    #
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--incognito') # incognito 시크릿 모드입니다.
    browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    #
    return browser
    

def quitBrowser(browser):
    #
    browser.quit()
    #
    return None

def getHtmlwithBrowser(browser, url):
    #
    browser.get(url)
    html = browser.page_source
    time.sleep(1)
    #
    return html

In [29]:
# 이미 수집된 html리스트를 읽는다 => html 이름으로 확인.

artist_html_path = "htmls/artists"
file_names = os.listdir(artist_html_path)
alread_crawled_artist_ids = set([_.replace(".html", "") for _ in file_names])

# 이미 parse된 artist를 읽는다.
try: 
    with open("artist_infos.jsonl", "r") as f:
        lines = f.readlines()
        docs = [json.loads(_) for _ in lines]
        parsed_artist_ids = [_["artist_num"] for _ in docs]
except:
    parsed_artist_ids = []


In [31]:
print("artist_id nums: {}".format(len(artist_ids)))
print("already crawled: {}".format(len(alread_crawled_artist_ids)))
print("already parsed: {}".format(len(parsed_artist_ids)))

artist_ids = [_ for _ in artist_ids if _ not in alread_crawled_artist_ids and _ not in parsed_artist_ids]
print("artist_ids to crawl: {}".format(len(artist_ids)))

artist_id nums: 7
already crawled: 0
already parsed: 2546
artist_ids to crawl: 7


In [32]:
#browser.quit()
browser = initBrowser()


for artist_id in tqdm(artist_ids):
    # https://www.melon.com/artist/timeline.htm?artistId=1034
    url = "https://www.melon.com/artist/detail.htm?artistId={}".format(artist_id)
    html = getHtmlwithBrowser(browser, url)
    
    
    if "잘못된 경로로 접근" in html:
        print("잘못된 경로", artist_id)
        browser.quit()
        browser = initBrowser()
        time.sleep(2)
        continue
        
    if artist_id not in html:
        print("artist_id가 없음.")
        continue
        
    #
    with open(os.path.join(artist_html_path, "{}.html".format(artist_id)), "w") as of:
        print(html, file=of)
        
    # 2초이상 쉬어야 끊기지 않는다.
    time.sleep(2)

[WDM] - Downloading: 100%|████████████████████████████████████| 8.81M/8.81M [00:00<00:00, 20.6MB/s]
/var/folders/03/p2hj2jdx6wq13p2tcscyfbwh0000gn/T/ipykernel_17336/4026465351.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
100%|████████████████████████████████████████████████████████████████| 7/7 [00:27<00:00,  3.92s/it]
